## Chapter 11
# Defining abstract data types

**11-0.** Compile execute, and test the programs in this chapter.

Implemented in `Vec.cpp` and `Vec.h`.

In [1]:
#include "Vec.h"

In [2]:
#include <string>

In [3]:
#include <iostream>

In [4]:
using std::string;

In [5]:
using std::cout;

In [6]:
using std::endl;

In [7]:
Vec<string> vec;
vec.push_back("hi");

(void) @0x70000e87aea0


In [8]:
vec[0];

(std::basic_string<char, std::char_traits<char>, std::allocator<char> > &) "hi"


**11-1.** The `Student_info` structure that we defined in Chapter 9 did not define a copy constructor, assignment operator, or destructor. Why not?

The `Student_info` structure did not define a copy constructor, assignment operator, or destructor because the synthesized implementation of these functions is sufficient for the class. (The synthesized implementations will apply these operations on each member variable recursively.)  The member variables are all primitives or collections of primitives, which do not require manual memory allocation or deallocation.

**11-2.** That structure did define a default constructor. Why?

Without a default constructor, when default-initialized the class would have otherwise had garbage values in its primitives, and in the primitives stored in its member vector.  (They would have held values based on whatever the contents of their memory happen to be.

**11-3.** What does the synthesized assignment operator for `Student_info` objects do?

As mentioned above, synthesized assignment operators for a class apply recursively to each member variable.  In the case of `Student_info` objects, the member variables `n`, `midterm`, `final`, `total_grade` and `homework` will all have the corresponding elements of the right-hand `Student_info` object assigned using the assignment operators defined for the `string`, `double` and `vector<double>` objects.  The assignment operators on each of these types copies their right-hand arguments to the left-hand variables.

**11-4.** How many members does the synthesized `Student_info` destructor destroy?

Only the members, `std::string n` and `std::vector<double> homework` will have synthesized destructors, since the built-in `double` type of the other members does technically have default destructors, but they have nothing to do (no memory to deallocate).

**11-5.** Instrument the `Student_info` class to count how often objects are created, copied, assigned and destroyed. Use this instrumented class to execute the student record programs from Chapter 6. Using the instrumented `Student_info` class will let you see how many copies the library algorithms are doing. Comparing the number of copies will let you estimate what proportion of the cost differences we saw are accounted for by the use of each library class. Do this instrumentation and analysis.

Here's how I tackled this:

First of all, the programs they're referring to are in _Chapter 5_, not Chapter 6!

I had a hard time sharing static instrumentation counter variables with the main process and the `Student_info` class, since the are compiled as separate objects and thus make separate copies of the static counters.  So I ended up putting three static counter variables (`copy_count`, `assign_count` and `destroy_count`) in `Student_info.h` and _also_ added a `void print_instrumentation();` function to the same header, defined in `Student_info.cpp`.  I call this print function from the main driver, and increment the variables appropriately in `Student_info.cpp`.

The extra methods look like:
```
Student_info::Student_info(const Student_info& other) {
    n = other.n;
    midterm = other.midterm;
    final = other.final;
    total_grade = other.total_grade;
    homework = other.homework;

    ++copy_count;
}

Student_info& Student_info::operator=(const Student_info& other) {
    if (this != &other) {
        n = other.n;
        midterm = other.midterm;
        final = other.final;
        total_grade = other.total_grade;
        homework = other.homework;
    }
    ++assign_count;
    return *this;
}

Student_info::~Student_info() {
    ++destroy_count;
}
```

I used the generically-typed `failing_students` code developed in a previous exercise so that I only had to change the initial `students` declaration between `vector` and `list`.

Here are the results (note that all of the remaining `destroy`s happen automatically _after_ user program termination, so the real number is much higher_:
```
$ gcc Student_info.cpp ../chapter_4/median.cpp failing_students_generic_type.cpp ../chapter_10/grade.cpp  -o failing_students_generic.out -lstdc++
```

**With list and size 1,000:**
```
Total copies: 1500
Total assignments: 0
Total destroys: 500
```
**With vector and size 1,000:**
```
Total copies: 3034
Total assignments: 250000
Total destroys: 2034
```

**With list and size 10,000:**
```
Total copies: 15000
Total assignments: 0
Total destroys: 5000
```
**With vector and size 10,000:**
```
Total copies: 39574
Total assignments: 25000000
Total destroys: 29574
```


It doesn't take much analysis to see that the number of copies only accounts for about 2X of the slowdown.  As `n` increases we see several orders of magnitude slowdown, which is clearly dominated by the explosive number of _assignments_ (of which the `list` implementation does none).

I note that with 100,000 students, the number of copies stays at about 2x in the vector impl, but the number of assignments is so large it cannot be contained in an `int`!

**11-6.** Add an operation to remove an element from a `Vec` and another to empty the entire `Vec`. These should behave analogously to the `erase` and `clear` operations on `vector`s.

In [9]:
vec.push_back("right");
vec.push_back("now");
vec.push_back("over");
vec.push_back("me");

(void) @0x70000e87aea0


In [10]:
vec.print_all();

hi
right
now
over
me


(void) @0x70000e87aea0


In [11]:
*(vec.end() - 1);

(std::basic_string<char, std::char_traits<char>, std::allocator<char> > &) "me"


In [12]:
vec.erase(vec.begin());

(Vec<std::__1::basic_string<char> >::iterator) 0x7f7fd17e8a90


In [13]:
vec.print_all();

right
now
over
me


(void) @0x70000e87aea0


In [14]:
*vec.begin();

(std::basic_string<char, std::char_traits<char>, std::allocator<char> > &) "right"


In [15]:
*(vec.end() - 1);

(std::basic_string<char, std::char_traits<char>, std::allocator<char> > &) "me"


In [16]:
vec.erase(vec.end() - 1);

(Vec<std::__1::basic_string<char> >::iterator) 0x7f7fd17e8ad8


In [17]:
vec.print_all();

right
now
over


(void) @0x70000e87aea0


In [18]:
*(vec.end() - 1);

(std::basic_string<char, std::char_traits<char>, std::allocator<char> > &) "over"


In [19]:
vec.clear();

(void) @0x70000e87aea0


In [20]:
vec.print_all();

(void) @0x70000e87aea0


**11-7.** Once you've added `erase` and `clear` to `Vec`, you can use that class insread of `vector` in most of the earlier programs in this book. Rewrite the `Student_info` programs from Chapter 9 and the programs that work with character pictures from Chapter 5 to use `Vec`s instead of `vector`s.

All copied over and adapted in the `vec_rewrites` folder.

(Turns out we really needed several more methods for drop-in replacement:

`Vec(const_iterator b, const_iterator e) { create(b, e); }`
`bool empty() { return end() == begin(); }`

I rewrote `insert` method calls to use `push_back` since they were just appending a bunch to the back.

**11-8.** Write a simplified version of the standard `list` class and its associated iterator.

All but iterators figured out in `List.h` and tested in `list_test.cpp`.

**11-9.** The `grow` function in 11.5.1/208 doubles the amount of memory each time it needs more. Estimate the efficiency gains of this strategy. Once you've predicted how much of a difference it makes, change the `grow` function appropriately and measure the difference.

The simplest alternative to doubling would be to simply grow space for one more element every time we run out.  This would incur the overhead of an `allocate`, `uninitialized_copy` call, along with a full `destroy` and `deallocate` call for the full `Vec` range, for each new element.

Compare this to a much larger `allocate`, `uninitialized_copy` copy much less frequently, with much less frequent but same-overhead calls to `destroy` and `deallocate`.

Assuming the call overhead of allocating memory is equal to the creation of an amount of memory of _any size_ (which is probably a bad assumption!), and assuming the penalty for each of `destroy` and `deallocate` for _any size_ are equal to the same time unit, the extra time would be something like 10-to-1.

Test runs:
**With double-style `grow`:**
```
$ gcc Student_info.cpp grade.cpp grading.cpp ../../chapter_4/median.cpp -o grading.out -lstdc++
$ time cat ../../chapter_5/grades100000 | ./grading.out
./grading.out  0.14s user 0.01s system 88% cpu 0.172 total
```
**With single-add-style `grow`:**
```
$ gcc Student_info.cpp grade.cpp grading.cpp ../../chapter_4/median.cpp -o grading.out -lstdc++
$ time cat ../../chapter_5/grades10000 | ./grading.out
./grading.out  10.64s user 0.06s system 99% cpu 10.714 total
```

Whewee! That's a big diff: 10.64s/0.14s = 76.0X

Running on size 100,000 wouldn't stop for as long as I was patient to wait for it, so it's pretty clear that the growth is exponential.